In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests
import folium

In [2]:
# mongoDB에 저장된 구군_ALL 목록을 얻어온다.
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')
starbucks = mongoConnection['starbucks']
gugunAll = starbucks['gugunAll']
gugunAllList = gugunAll.find({}, {'_id': 0, 'gugunAll_cd': 1, 'gugunAll_nm': 1})
# for s in gugunAllList:
    # print(s)

In [3]:
sidoName = input('검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
sido_cd = gugunAll.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
print(sido_cd['sido_cd'])


검색하려는 시도 이름을 입력하세요(예: 서울): 서울
01


In [4]:
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=3112E6S5DE'
req = requests.post(targetSite, data = {
    'ins_lat': 37.5717888, # 위도
    'ins_lng': 126.9661696, # 경도
    'p_sido_cd': sido_cd['sido_cd'], # 시도코드
    #'p_gugun_cd':gugun_cd['gugun_cd'], # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
    # in_biz_cd, set_date를 넘겨주지 않으면 서버에서 데이터를 얻어오지 못한다.
})
storeList = req.json()
print(len(storeList['list']))

count = 0
for store in storeList['list']:
    count += 1
    #print('{:4d}. {}: {}({}, {})'.format(count, store['s_name'], store['doro_address'], store['lat'], store['lot']))


599


In [5]:
starbucksMap = folium.Map(location=[36.56009115, 127.83364774], zoom_start=7)

count = 0
for store in storeList['list']:
    count += 1
    popup = folium.Popup(store['s_name']+'점', max_width=300)
    folium.Marker(location=[store['lat'], store['lot']], popup=popup).add_to(starbucksMap)

starbucksMap